## Proposta

Encontrar os valores dos campos de controle, $f_x$, $f_y$ e $f_z$. Os valores de controle podem ser definidos como: $$f_{x,y,z}=-a_{x,y,z}S(A_{x,y,z},t) - h_{x,y,z}\frac{S(B,t)}{S(A_{x,y,z},t)}$$

$S$ é a representação do controle de Lyapunov já que este consiste nas componentes que formam a derivada de $V(t)$, cujo o propósito, em Lyapunov, é ser menor que zero enquanto $V(t)$ deve ser maior que zero, em qualquer tempo. $S$ pode ser definido como: $$S(X,t)=(-\frac{1}{2}U_t^{\dagger}X^{\dagger}U_fW^{\dagger}-\frac{1}{2}W^{\dagger}U_t^{\dagger}X^{\dagger}U_f+2U_t^{\dagger}X^{\dagger}U_f)L$$

Para começar os trabalhos, iniciamos em $t=0$ pois como $U$ é a evolução temporal unitária, $U_0 = diag(1,1,1)$, ademais, $X \in \{A_x, A_y, A_z, B\}$, $U_f=\left(\begin{array}{cc} 
0 & 0 & 1\\
0 & -1 & 0\\
1 & 0 & 0\\
\end{array}\right)$, $h_x+h_y+h_z = 1$. O $A's$ e $B$ são matrizes que foram obtidas por meio da transformação adjunta em $SO(3)$ da equação mestra de Lindblad, por fim, $W_t = U_f^{\dagger}U_t$ e por consequência $L_t = W_t - I -\frac{1}{2}(W_t-I)^2$.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import random

In [2]:
def lyap(Ui, Uf, X):
    first=-1/2*np.matmul(np.matmul(np.matmul(Ui.getH(),X.getH()),Uf),np.matmul(Uf.getH(), Ui).getH())
    second=-1/2*np.matmul(np.matmul(Uf.getH(), Ui).getH(), np.matmul(np.matmul(Ui.getH(),X.getH()),Uf))
    third=2*np.matmul(np.matmul(Ui.getH(),X.getH()),Uf)
    L=np.subtract(np.matmul(Uf.getH(), Ui), np.identity(3))-1/2*np.subtract(np.matmul(Uf.getH(), Ui), np.identity(3))**2
    return 2*np.trace((first+second+third)*L)
    #return (first+second+third)*L

def control(Ui, Uf, a, h, A, B):
    SA=lyap(Ui, Uf, A)
    SB=lyap(Ui, Uf, B)
    return -a*SA-h*(SB/SA)

# U' = (A(t)+B)U
def dUdt(u, Ui, Uf, t):
    Ax = np.matrix('0 0 0; 0 0 -1; 0 1 0')
    Ay = np.matrix('0 0 1; 0 0 0; -1 0 0')
    Az = np.matrix('0 -1 0; 1 0 0; 0 0 0')
    B = 0.01*np.matrix('-1 0 0; 0 -1 0; 0 0 -2')
    if t == 0:
        F = np.dot(10.28, Ax)+np.dot(10.73, Ay)+np.dot(40, Az)
    else:
        fx = control(Ui, Uf, u[0], u[3], Ax, B)
        fy = control(Ui, Uf, u[1], u[4], Ay, B)
        fz = control(Ui, Uf, u[2], u[5], Az, B)
        print(f"Control values -> fx = {fx} fy = {fy} fz = {fz}")
        F = np.dot(fx, Ax)+np.dot(fy, Ay)+np.dot(fz,Az)+B
    return np.matmul(F, Ui)

# Runge-Kutta de quarta ordem

def runge_kutta(u, Ui, Uf, h, t):
    
    k1 = dUdt(u, Ui, Uf, t)
    k2 = dUdt(u, Ui + np.dot((0.5*h), k1), Uf, t)
    k3 = dUdt(u, Ui + np.dot((0.5*h), k2), Uf, t)
    k4 = dUdt(u, Ui + np.dot(h, k3), Uf, t)
    
    A = np.dot(2,k3) + k4
    B = np.dot(2,k2) + k1
    C = A+B

    res = Ui + np.dot((h / 6.0),(C))
        
    return res

In [3]:
def func_objetivo(u, Ui, Uf, h, t):
    return np.linalg.norm(np.subtract(Uf, runge_kutta(u, Ui, Uf, h, t)))

In [4]:
def const(u): #h_x+h_y+h_z=1
    return u[3] + u[4] + u[5] - 1    

def main():
    
    # Instância
    cons = {'type':'eq', 'fun': const}
    u = []
    # a's
    u.append(70)
    u.append(106)
    u.append(66)
    
    #h's
    u.append(0.35)
    u.append(0.31)
    u.append(0.34)

    h = 0.001 #Runge-kutta

    t = 0

    Ui = np.matrix('1 0 0; 0 1 0; 0 0 1')
    Uf = np.matrix('0 0 1; 0 -1 0; 1 0 0')
    print(f'U_{t}: {Ui}')
    print(f'Fidelity: {(np.trace(np.matmul(Ui, Ui.getH()))+np.abs(np.trace(np.matmul(Uf.getH(), Ui)))**2)/(3*(3+1))}')
    print(f'Distance: {np.trace((Ui - Uf).getH()*(Ui - Uf))}')

    while t < 0.1:

        op = optimize.minimize(func_objetivo, u, args=(Ui, Uf, h, t), constraints=cons, method='SLSQP')
        print(f'The optimization result is: {op.x}')
        u = op.x
        Ui = runge_kutta(u, Ui, Uf, h, t)
        print(f'U_{t}: {Ui}')
        print(f'Fidelity: {(np.trace(np.matmul(Ui, Ui.getH()))+np.abs(np.trace(np.matmul(Uf.getH(), Ui)))**2)/(3*(3+1))}')
        print(f'Distance: {np.trace((Ui - Uf).getH()*(Ui - Uf))}')

        t = round(t+0.001, 3)

    

In [5]:
main()

U_0: [[1 0 0]
 [0 1 0]
 [0 0 1]]
Fidelity: 0.3333333333333333
Distance: 8
The optimization result is: [ 70.   106.    66.     0.35   0.31   0.34]
U_0: [[ 0.99914256 -0.03993272  0.01093231]
 [ 0.04004301  0.99914729 -0.01006231]
 [-0.01052117  0.01049145  0.99988961]]
Fidelity: 0.33312282519489606
Distance: 7.997472305028497
Control values -> fx = 49.212988229225346 fy = -16.708102778237034 fz = 45.89091320178277
Control values -> fx = 95.77055575087213 fy = -7.30246557384978 fz = 89.78449270116566
Control values -> fx = 139.7298066763416 fy = -3.9776729987212196 fz = 131.38794741778975
Control values -> fx = 311.99215220474093 fy = 2.137985569703506 fz = 292.8787206275834
Control values -> fx = 49.2129882397648 fy = -16.708102778237034 fz = 45.89091320178277
Control values -> fx = 95.7705557765058 fy = -7.302465574430544 fz = 89.78449270604123
Control values -> fx = 139.72980672094062 fy = -3.977673001402837 fz = 131.38794743181543
Control values -> fx = 311.99215232700976 fy = 2.1379